In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

train = pd.read_csv('Data/train.csv')

train.head(15)

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1
5,1.2.826.0.1.3680043.8.498.11707076266253086830...,0,0,1,0,0,0,1,0,0,1,0,8260896cf
6,1.2.826.0.1.3680043.8.498.23133310896792275593...,0,0,0,0,0,0,0,0,1,1,0,f6147af3f
7,1.2.826.0.1.3680043.8.498.17332833449968516367...,0,0,1,0,0,0,1,0,1,0,0,27413587b
8,1.2.826.0.1.3680043.8.498.91266853519846682206...,0,0,0,0,0,1,0,0,0,1,0,2edf3dce3
9,1.2.826.0.1.3680043.8.498.12706894217669234973...,0,1,0,0,0,0,1,0,0,1,0,3d5e7832e
